<a href="https://colab.research.google.com/github/click-b8/Gyro-Stabilization-Platform-/blob/main/Active_Maritime_Stabilization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Controller Class

In [1]:
import time
import math

class PlatformController:
    def __init__(self, Kp, Ki, Kd, Kff, dt):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.Kff = Kff
        self.dt = dt

        self.integral = 0.0
        self.prev_error = 0.0

    def update(self, theta_meas, theta_pred):
        # Desired angle = 0 (level platform)
        error = 0.0 - theta_meas

        # Integral term
        self.integral += error * self.dt

        # Derivative term
        derivative = (error - self.prev_error) / self.dt

        # PID feedback
        u_fb = (self.Kp * error +
                self.Ki * self.integral +
                self.Kd * derivative)

        # Feedforward (wave prediction)
        u_ff = -self.Kff * theta_pred

        # Total control output
        u = u_fb + u_ff

        self.prev_error = error

        return u


In [ ]:
# Lightweight Wave Prediction

In [ ]:
def predict_wave(A, omega, phi, t_future):
    return A * math.sin(omega * t_future + phi)


In [ ]:
# Main Reltime-Loop

In [ ]:
controller = PlatformController(
    Kp=8.0,
    Ki=0.5,
    Kd=2.5,
    Kff=1.2,
    dt=0.01  # 100 Hz
)

t0 = time.time()

while True:
    t = time.time() - t0

    # ---- 1. Read sensors ----
    theta_meas = get_platform_angle()      # from IMU fusion
    theta_dot  = get_platform_rate()       # gyro

    # ---- 2. Get wave model parameters (from FFT block) ----
    A, omega, phi = get_wave_parameters()

    # Predict 0.5 sec ahead
    theta_pred = predict_wave(A, omega, phi, t + 0.5)

    # ---- 3. Run control law ----
    motor_cmd = controller.update(theta_meas, theta_pred)

    # ---- 4. Send to motor driver ----
    set_motor_torque(motor_cmd)

    time.sleep(0.01)


In [ ]:
# Tuning Guide

In [ ]:
Kff = 0
Ki = 0



In [ ]:
# Step 1: Tune K_p

# Increase until platform resists tilt strongly but doesn’t oscillate.

# Step 2 — Add Kd

# Increase until motion becomes smooth and stops overshooting.

# Step 3 — Add Small Ki

# Only enough to remove slow drift.

#Step 4 — Turn on Feedforward

# Slowly increase Kff (0 → 1.5)
# You’ll see lag disappear and motors move before the wave hits.
